### Основная идея:

За основу взята модель RoSBERTa (["ai-forever/ru-en-RoSBERTa"](https://huggingface.co/ai-forever/ru-en-RoSBERTa) on Huggingface).
Решение взять ансамбль из 3-ех лучших моделей RoSBERTa, где менялись гиперпараметры  для данных (эксперименты), и эти результаты усреднить.  
<br>
Пример:  

| id | tweet      | class |  
|:---|:----------:|------:|
| 03 | some_tweet |   1   |

model_1 выдает предикт = 1  
model_2 выдает предикт = 0  
model_3 выдает предикт = 1  

Считаем: (1 + 0 + 1) / 3 = 0.67, далее округляем по мат. правилам => 1, записываем в сабмишн 1.   

<br>

### Детали и тонкости:

1) Обработка данных:  
Почистил твиты от мусора и шума (убрал @usernames, ссылки и лишние пробелы). Из-за @usernames длина некоторых твитов была очень большой.  

2) Балансировка данных (smart undersampling):
В тренировочном датасете был дисбаланс ~1:9 положительных (class = 1) к отрицательным (class = 0).
Чтобы модель не была предвзятой к отрицательному классу и не склонялась к его выбору, я уменьшил количество отрицательных примеров в тренировочном наборе, установив разные пропорции ~ (1:3, 1:3.5, 1:4) между положительными и отрицательными примерами. Это позволяет моделям видеть достаточное количество примеров обоих классов и лучше трениться.   
А в валидационном наборе я сохранил оригинальный дисбаланс данных, чтобы метрика качества F1-score рассчитывалась в условиях, приближенных к реальности, где много отрицательных примеров.

3) Оптимизация F1-score (подбор порогового значения):  
Вычислял оптимального трешхолда, вместо дефолтного 0.5, для максимизации F1-метрики в функции `compute_metrics`.  

<br>

### Вывод

Усреднение значений моделей (ансамбль), дает лучшую оценку, нежели каждая модель по отдельности.  

Так же можно было поиграть с весами самой модели, но как-то руки не дошли)

In [59]:
!wget -q -O train.tsv https://www.dropbox.com/s/2nvhmusyozfrrn9/train.tsv?dl=0
!wget -q -O test.tsv https://www.dropbox.com/s/77s33v3q3q1i5mr/test.tsv?dl=0

In [2]:
!pip install -q transformers datasets bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import os
import re
import torch
import numpy as np
import pandas as pd

from torch import nn
from datasets import Dataset
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)

from sklearn.metrics import (
    precision_recall_fscore_support,
    accuracy_score,
    f1_score
)

from torch.utils.data import (
    DataLoader,
    WeightedRandomSampler
)

### Анализ и предобработка

In [4]:
train_df = pd.read_csv("train.tsv", sep=",")
test_df = pd.read_csv("test.tsv", sep=",")

In [5]:
print(train_df.shape, test_df.shape)
train_df.head()

(9515, 3) (1504, 2)


,id,tweet,class
0,760402871867367424,"Настало время для ингаляторов. Дружок, Сальбут...",0
1,1035908416869462016,15) На прошлой зимней олимпиаде большинство лы...,1
2,1089839736427032577,Не соглашусь с заменой ЗОК на метопролол в так...,0
3,779671488748224513,"@di2m1 мезим Смекта Если отравление, то лоперамид",0
4,738309299756240897,Уберите микроволновки и имодиум Действуют соу...,0


In [6]:
train_df["tweet"].str.len().describe()

,tweet
count,9515.000000
mean,115.221755
std,69.618444
min,1.000000
25%,61.000000
50%,98.000000
75%,143.000000
max,390.000000


In [7]:
train_df["tweet"].str.len().quantile(0.99)

280.8600000000006

In [8]:
for ind, row in train_df[train_df["tweet"].str.len() > 300].head().iterrows():
  print(f"Ind: {ind} =================")
  print(f"Class: {row['class']}")
  print(f"Tweet: {row['tweet']}")

Ind: 279 =================
Class: 0
Tweet: @moskvit1960 @ChoShto Я ни за что не стану с Вами спорить. 850Вы всегда этого хотите. А я не люблю"боевые действия". У меня мама 6,5 лет назад умерла в возрасте 70,5лет До этого она жила 10 л. слепой и с порезом желудка.У неё было 5-6 уколов в день!!! И 3 р.в день1/2 Метформин 850 - я знаю об этом ВСЁ
Ind: 290 =================
Class: 0
Tweet: @fulaankungen @aidsexpert Оксикодон имеет  положительный эффект– возможность подавления абстинентной симптоматики от других наркотических средств, принимаемых наркоманом.Но при этом препарат не используется при лечении зависимости –согласно мнению специалистов, от него очень быстро развивается зависимость
Ind: 626 =================
Class: 1
Tweet: @xsmmooxxxoo @prima_bezfiltra Нет. Мне прописали нейролептик Кветиапин (Сероквель) и антидепрессант Тразодон (Триттико). Значительно улучшилось засыпание, я перестал просыпаться ночью, уменьшилась тревожность. Из минусов: тяжело просыпаться не по времени. Но, к

In [9]:
def preprocess_tweet(tweet: str) -> str:
    """
    Предобработка твитов (очищение от шума).

    Этот метод удаляет ненужную информацию из текста твита, включая упоминания, ссылки и лишние пробелы.

    Args:
        tweet (str): Исходный текст твита.

    Returns:
        str: Очищенный текст твита.
    """
    tweet = re.sub(r"@\w+", "", tweet)             # Удаление @usernames
    tweet = re.sub(r"http\S+|www\S+", "", tweet)   # Удаление ссылок
    tweet = re.sub(r"\s+", " ", tweet).strip()     # Удаление лишних пробелов
    return tweet

In [10]:
# Предобработка твитов в датафрейме
train_df["tweet"] = train_df["tweet"].astype(str).apply(preprocess_tweet)
test_df["tweet"] = test_df["tweet"].astype(str).apply(preprocess_tweet)

In [11]:
train_df["tweet"].str.len().describe()

,tweet
count,9515.000000
mean,112.634997
std,68.525066
min,1.000000
25%,59.000000
50%,95.000000
75%,140.000000
max,288.000000


In [12]:
train_df["tweet"].str.len().quantile(0.99)

279.0

In [13]:
for ind, row in train_df[train_df["tweet"].str.len() > 250].head().iterrows():
  print(f"Ind: {ind} =================")
  print(f"Class: {row['class']}")
  print(f"Tweet: {row['tweet']}")

Ind: 1 =================
Class: 1
Tweet: 15) На прошлой зимней олимпиаде большинство лыжников приехало со справкой о том что у них якобы астма. Сделано это было для того, чтобы легально принимать сальбутамол (то же что и я принимаю в ингаляторах) который расширяет объём легких. По сути допинг для здорового человека.
Ind: 57 =================
Class: 0
Tweet: В 29 лет нетривиальной задачей становится собрать аптечку в поездку по городам. Пока что ограничился следующим: Миг 400 Септолете тотал Назол Микролакс Гилан Прозак Имован Атаракс Феназепам Драмина Спирт Пластыри Антигистаминные брать не стал, всё таки зима. Что забыл?
Ind: 58 =================
Class: 0
Tweet: Я сегодня слышала от мам чего хотят мужчины из порно стоящие или проезжающие рядом: «этот хочет чтоб обсасывала сперму, а тот нет прикинь» мне кажется надо по две пить прозака, последний раз я слышала чужим голосом потому что своим голосом такое слушать страшно
Ind: 60 =================
Class: 1
Tweet: Огосподи, ну если «не ес

### Балансировка данных

In [14]:
np.unique(train_df["class"], return_counts=True)

(array([0, 1]), array([8683,  832]))

In [15]:
n_positive_samples = (train_df["class"] == 1).sum() # ~ 832
n_negative_samples = (train_df["class"] == 0).sum() # ~ 8683

In [16]:
def balancing_dataset(rs: int, cap_coef: float) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Балансировка датасета методом undersampling.

    Этот метод формирует сбалансированные тренировочный и валидационный наборы данных,
    уменьшая количество негативных примеров, чтобы они соответствовали числу позитивных примеров.

    Args:
        rs (int): Значение random_state для воспроизводимости разбиения данных.
        cap_coef (float): Коэффициент, определяющий количество негативных примеров в тренировочном наборе
            относительно количества позитивных примеров.

    Returns:
        tuple[pd.DataFrame, pd.DataFrame]:
            - train_data (pd.DataFrame): Сбалансированный тренировочный набор данных.
            - val_data (pd.DataFrame): Сбалансированный валидационный набор данных.
    """
    train_positive, val_positive = train_test_split(
        train_df[train_df["class"] == 1],
        test_size=0.2,
        random_state=rs
    )

    n_positive_train = train_positive.shape[0] # ~ 665
    n_positive_val = val_positive.shape[0] # ~ 167

    n_negative_train = int(cap_coef * n_positive_train)
    n_negative_val = int(n_positive_val / n_positive_samples * n_negative_samples)

    trainval_negative = train_df[train_df["class"] != 1].sample(
        n_negative_train + n_negative_val,
        random_state=rs
    )

    train_negative = trainval_negative[:n_negative_train]
    val_negative = trainval_negative[n_negative_train:]

    train_data = pd.concat([train_positive, train_negative])
    val_data = pd.concat([val_positive, val_negative])

    return train_data, val_data

### Токенизация

In [17]:
model_name = "ai-forever/ru-en-RoSBERTa"

In [18]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.49M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.99M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

In [19]:
tokenizer("Hello world", truncation=True)

{'input_ids': [1, 79891, 49370, 2], 'attention_mask': [1, 1, 1, 1]}

In [20]:
def tokenize_function(examples) -> dict:
    """
    Токенизация твитов.

    Этот метод принимает на вход набор текстов (твитов) и возвращает токенизированные данные,
    подготовленные для использования в модели.

    Args:
        examples (dict): Словарь с ключом "tweet", содержащий список строк (текстов твитов).

    Returns:
        dict: Токенизированные данные, возвращаемые токенизатором. Каждый текст будет усечён до
              максимально допустимой длины (truncation=True).
    """
    return tokenizer(examples["tweet"], truncation=True)

In [21]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Функции оптимизация F1-score и подсчет метрик

In [22]:
def optimize_f1(labels: np.ndarray, predictions: np.ndarray) -> tuple[float, float]:
    """
    Оптимизация F1-метрики путём подбора наилучшего порога для вероятностей предсказаний.

    Args:
        labels (np.ndarray): Истинные метки класса (0 или 1).
        predictions (np.ndarray): Логиты (сырые выходы модели) для каждого класса.

    Returns:
        tuple[float, float]:
            - best_f1 (float): Максимально достигнутое значение F1-метрики.
            - best_threshold (float): Порог, при котором достигается лучший F1.
    """
    probs = nn.functional.softmax(torch.tensor(predictions), dim=1)

    probs_positive = probs[:, 1].detach().cpu().numpy()

    best_f1 = 0
    best_threshold = 0

    for p in np.sort(probs_positive)[1:-1]:
        predicted_labels = (probs_positive >= p).astype(int)
        f1 = f1_score(labels, predicted_labels)
        if f1 > best_f1:
        best_f1 = f1
        best_threshold = p

    return best_f1, best_threshold

In [23]:
def compute_metrics(pred: transformers.EvalPrediction) -> dict:
    """
    Вычисление метрик для предсказаний модели, включая F1-метрику, точность, полноту и оптимальный порог.

    Args:
        pred: Объект с двумя атрибутами:
            - label_ids (np.ndarray): Истинные метки классов.
            - predictions (np.ndarray): Логиты (сырые предсказания модели).

    Returns:
        dict: Словарь с вычисленными метриками, включая:
            - "accuracy" (float): Точность предсказаний.
            - "f1" (float): F1-метрика.
            - "precision" (float): Точность.
            - "recall" (float): Полнота.
            - "threshold" (float): Оптимальный порог для F1-метрики.
    """
    labels = pred.label_ids
    preds = pred.predictions

    best_f1, best_threshold = optimize_f1(labels, preds)
    pred_labels = (preds[:, 1] >= best_threshold).astype(int)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred_labels, average="binary")
    acc = accuracy_score(labels, pred_labels)

    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
        "threshold": best_threshold
    }

### Создание датасетов, настройка и обучение модели

#### Model 1
###### random_state = 1
###### capacity_coefficient = 3

In [24]:
train_data_1, val_data_1 = balancing_dataset(1, 3)

train_dataset_1 = Dataset.from_pandas(train_data_1).map(tokenize_function, batched=True)
val_dataset_1 = Dataset.from_pandas(val_data_1).map(tokenize_function, batched=True)
test_dataset = Dataset.from_pandas(test_df).map(tokenize_function, batched=True)

Map:   0%|          | 0/2660 [00:00<?, ? examples/s]

Map:   0%|          | 0/1909 [00:00<?, ? examples/s]

Map:   0%|          | 0/1504 [00:00<?, ? examples/s]

In [25]:
train_dataset_1 = train_dataset_1.rename_column("class", "labels")
val_dataset_1 = val_dataset_1.rename_column("class", "labels")

In [26]:
model_1 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.61G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ru-en-RoSBERTa and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
model_1

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(98505, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=Tru

In [28]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.1,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none",
)

In [29]:
trainer = Trainer(
    model=model_1,
    args=training_args,
    train_dataset=train_dataset_1,
    eval_dataset=val_dataset_1,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Threshold
1,No log,0.238275,0.924568,0.560976,0.571429,0.550898,0.752016
2,0.352700,0.336406,0.918282,0.566667,0.528497,0.610778,0.947314
3,0.352700,0.674654,0.882137,0.507659,0.400000,0.694611,0.998285
4,0.184800,0.749604,0.885804,0.526087,0.412969,0.724551,0.999349
5,0.055900,0.758055,0.885280,0.512249,0.407801,0.688623,0.999635


TrainOutput(global_step=1665, training_loss=0.18090196586585974, metrics={'train_runtime': 1363.2568, 'train_samples_per_second': 9.756, 'train_steps_per_second': 1.221, 'total_flos': 1591272802882560.0, 'train_loss': 0.18090196586585974, 'epoch': 5.0})

In [31]:
val_preds_1 = trainer.predict(val_dataset_1).predictions
val_labels_1 = val_dataset_1["labels"]

best_f1, best_threshold = optimize_f1(val_labels_1, val_preds_1)
best_f1, best_threshold

(0.5644171779141104, 0.94731385)

In [32]:
predictions_1 = trainer.predict(test_dataset)

In [33]:
probs_1 = nn.functional.softmax(torch.tensor(predictions_1.predictions), dim=1)

In [34]:
print(f"Используем порог: {best_threshold}")
test_df["class"] = (probs_1[:, 1] >= best_threshold).int()

submission_path = "model_1.csv"
test_df[["id", "class"]].to_csv(submission_path, index=False)
print(f"Файл сохранён: {submission_path}")

Используем порог: 0.9473138451576233
Файл сохранён: model_1.csv


#### Model 2
###### random_state = 42
###### capacity_coefficient = 4

In [35]:
train_data_2, val_data_2 = balancing_dataset(42, 4)

train_dataset_2 = Dataset.from_pandas(train_data_2).map(tokenize_function, batched=True)
val_dataset_2 = Dataset.from_pandas(val_data_2).map(tokenize_function, batched=True)
test_dataset = Dataset.from_pandas(test_df).map(tokenize_function, batched=True)

Map:   0%|          | 0/3325 [00:00<?, ? examples/s]

Map:   0%|          | 0/1909 [00:00<?, ? examples/s]

Map:   0%|          | 0/1504 [00:00<?, ? examples/s]

In [36]:
train_dataset_2 = train_dataset_2.rename_column("class", "labels")
val_dataset_2 = val_dataset_2.rename_column("class", "labels")

In [37]:
model_2 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ru-en-RoSBERTa and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
model_2

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(98505, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=Tru

In [39]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.1,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none",
)

In [40]:
trainer = Trainer(
    model=model_2,
    args=training_args,
    train_dataset=train_dataset_2,
    eval_dataset=val_dataset_2,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

In [41]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Threshold
1,No log,0.158325,0.942378,0.623288,0.728000,0.544910,0.325553
2,0.346300,0.248454,0.937664,0.631579,0.653846,0.610778,0.709188
3,0.204300,0.411655,0.919329,0.620690,0.527197,0.754491,0.987356
4,0.094200,0.576033,0.906234,0.599553,0.478571,0.802395,0.998809
5,0.047200,0.465250,0.927187,0.638961,0.564220,0.736527,0.997413


TrainOutput(global_step=2080, training_loss=0.1683871003297659, metrics={'train_runtime': 1635.3629, 'train_samples_per_second': 10.166, 'train_steps_per_second': 1.272, 'total_flos': 1973475692732124.0, 'train_loss': 0.1683871003297659, 'epoch': 5.0})

In [42]:
val_preds_2 = trainer.predict(val_dataset_2).predictions
val_labels_2 = val_dataset_2["labels"]

best_f1, best_threshold = optimize_f1(val_labels_2, val_preds_2)
best_f1, best_threshold

(0.6764705882352942, 0.997413)

In [43]:
predictions_2 = trainer.predict(test_dataset)

In [44]:
probs_2 = nn.functional.softmax(torch.tensor(predictions_2.predictions), dim=1)

In [45]:
print(f"Используем порог: {best_threshold}")
test_df["class"] = (probs_2[:, 1] >= best_threshold).int()

submission_path = "model_2.csv"
test_df[["id", "class"]].to_csv(submission_path, index=False)
print(f"Файл сохранён: {submission_path}")

Используем порог: 0.9974129796028137
Файл сохранён: model_2.csv


#### Model 3
###### random_state = 1
###### capacity_coefficient = 3.5

In [46]:
train_data_3, val_data_3 = balancing_dataset(1, 3.5)

train_dataset_3 = Dataset.from_pandas(train_data_3).map(tokenize_function, batched=True)
val_dataset_3 = Dataset.from_pandas(val_data_3).map(tokenize_function, batched=True)
test_dataset = Dataset.from_pandas(test_df).map(tokenize_function, batched=True)

Map:   0%|          | 0/2992 [00:00<?, ? examples/s]

Map:   0%|          | 0/1909 [00:00<?, ? examples/s]

Map:   0%|          | 0/1504 [00:00<?, ? examples/s]

In [47]:
train_dataset_3 = train_dataset_3.rename_column("class", "labels")
val_dataset_3 = val_dataset_3.rename_column("class", "labels")

In [48]:
model_3 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ru-en-RoSBERTa and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
model_3

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(98505, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=Tru

In [50]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.1,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none",
)

In [51]:
trainer = Trainer(
    model=model_3,
    args=training_args,
    train_dataset=train_dataset_3,
    eval_dataset=val_dataset_3,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

In [52]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Threshold
1,No log,0.203192,0.932949,0.549296,0.666667,0.467066,0.617150
2,0.346100,0.400018,0.920377,0.582418,0.538071,0.634731,0.613311
3,0.194600,0.635301,0.888423,0.535948,0.421233,0.736527,0.993115
4,0.194600,0.688277,0.900995,0.559441,0.458015,0.718563,0.996496
5,0.086000,0.744609,0.895757,0.550790,0.442029,0.730539,0.997594


TrainOutput(global_step=1870, training_loss=0.17389401012563452, metrics={'train_runtime': 1481.276, 'train_samples_per_second': 10.099, 'train_steps_per_second': 1.262, 'total_flos': 1763359753692096.0, 'train_loss': 0.17389401012563452, 'epoch': 5.0})

In [53]:
val_preds_3 = trainer.predict(val_dataset_3).predictions
val_labels_3 = val_dataset_1["labels"]

best_f1, best_threshold = optimize_f1(val_labels_3, val_preds_3)
best_f1, best_threshold

(0.6005221932114883, 0.61331135)

In [54]:
predictions_3 = trainer.predict(test_dataset)

In [55]:
probs_3 = nn.functional.softmax(torch.tensor(predictions_3.predictions), dim=1)

In [56]:
print(f"Используем порог: {best_threshold}")
test_df["class"] = (probs_3[:, 1] >= best_threshold).int()

submission_path = "model_3.csv"
test_df[["id", "class"]].to_csv(submission_path, index=False)
print(f"Файл сохранён: {submission_path}")

Используем порог: 0.6133113503456116
Файл сохранён: model_3.csv


### Вычисление среднего значения всех 3 моделей

In [57]:
file_paths = [
    'model_1.csv',
    'model_2.csv',
    'model_3.csv'
]

output_file = 'submission.csv'

In [58]:
dfs = [pd.read_csv(file_path) for file_path in file_paths]

for i, df in enumerate(dfs):
    if not {'id', 'class'}.issubset(df.columns):
        raise ValueError(f"File {file_paths[i]} does not have the required columns.")

merged_df = dfs[0][['id']]
merged_df['class'] = np.mean([df['class'] for df in dfs], axis=0)

merged_df['class'] = merged_df['class'].round().astype(int)

merged_df.to_csv(output_file, index=False)
print(f"Усреднённые значения сохранены в файл: {output_file}")

Усреднённые значения сохранены в файл: submission.csv
